<a href="https://colab.research.google.com/github/VidhyaSree-N/classifier_synthetic_original_mammograms/blob/main/Classifierwith512_512data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
%pylab inline
import numpy as np
import os
import cv2

Populating the interactive namespace from numpy and matplotlib


In [2]:
#
# 1. Load Data
#

train_image_folder = '/content/drive/MyDrive/512data/train/AI'
train_npz_folder = '/content/drive/MyDrive/512data/train/original'

test_image_folder = '/content/drive/MyDrive/512data/test/AI'
test_npz_folder ='/content/drive/MyDrive/512data/test/original'

In [3]:
img_height = 512
img_width = 512
batch_size = 32

In [11]:
def custom_data_generator(image_folder, npz_folder, batch_size, img_height, img_width):
  num_images_loaded = 0
  while True:
        image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
        npz_files = [f for f in os.listdir(npz_folder) if f.endswith('.npz')]

        total_files = min(len(image_files), len(npz_files))
        indices = np.arange(total_files)
        np.random.shuffle(indices)

        for i in range(0, total_files, batch_size):
            batch_indices = indices[i:i + batch_size]

            batch_images = []
            batch_npz = []

            for idx in batch_indices:
                img_file = os.path.join(image_folder, image_files[idx])
                npz_file = os.path.join(npz_folder, npz_files[idx])

                image = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (img_width, img_height))

                with np.load(npz_file, allow_pickle=True) as data:
                    npz = data['data']
                npz = np.resize(npz, (img_height, img_width))

                batch_images.append(image)
                batch_npz.append(npz)
                num_images_loaded += 1

            if len(batch_images) > 0:
                print(f"Loaded {num_images_loaded} images into batch")
                print(f"Batch Image Shape: {np.array(batch_images).shape}")
                print(f"Batch NPZ Shape: {np.array(batch_npz).shape}")
                yield (np.array(batch_images), np.array(batch_npz))


In [5]:
# train_generator = custom_data_generator(train_image_folder, train_npz_folder, batch_size, img_height, img_width)

In [5]:
test_generator = custom_data_generator(test_image_folder,test_npz_folder, batch_size, img_height, img_width)

In [6]:
NUMBER_OF_CLASSES = 2

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3),
                             activation='relu',
                             input_shape=(img_height, img_width, 1)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(NUMBER_OF_CLASSES, activation='softmax'))


In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
# Create the data generators
train_generator = custom_data_generator(train_image_folder, train_npz_folder, batch_size, img_height, img_width)

try:
    model.fit(
        train_generator,
        epochs=1,
        steps_per_epoch=None,
        verbose=1
    )
except Exception as e:
    print("An error occurred:", str(e))



Loaded 32 images into batch
Batch Image Shape: (32, 512, 512)
Batch NPZ Shape: (32, 512, 512)
Loaded 64 images into batch
Batch Image Shape: (32, 512, 512)
Batch NPZ Shape: (32, 512, 512)
Loaded 96 images into batch
Batch Image Shape: (32, 512, 512)
Batch NPZ Shape: (32, 512, 512)
